# Project 2

## Setup


- Create project directory:
```
mkdir project-2-eliu390
cd project-2-eliu390
```


- Download assessment data:
```
curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp
```


- Spin up Docker containers:
```
docker-compose up -d
```


- Container status check:
```
docker-compose logs -f kafka
docker-compose ps
```


- Create Kafka topic "assessments":
```
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```


- Kafka topic status check:
```
docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181
```


- Publish assessment data as Kafka event under "assessments" topic:
```
docker-compose exec mids bash -c "cat /w205/project-2-eliu390/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"
```


- Run Pyspark in Jupyter instance:
```
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root --notebook-dir=/w205/' pyspark
```


- Create Firewall Rule under "VPC Network" for GCP AI Platform notebook:
    - Allow IP range: 0.0.0.0/0
    - Allow port: 8888
 
 
- Navigate to Pyspark Jupyter instance URL
    - Replace "0.0.0.0" with GCP notebook external IP address (found in GCP Compute Engine)
    - Note: instance URL and GCP notebook IP address change upon restart

## Extract, Transform, Load Data

In [1]:
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType, ArrayType
import warnings

The assessments data was manually published using the command line as a Kafka event, under the topic "assessments". We now connect Spark to Kafka, subscribe to the "assessments" topic, and request all of the messages by setting a broad starting and ending offset for events.

In [2]:
raw_assessments = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","assessments") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 

Having Spark cache the raw data saves it in memory, which speeds up the next few exploratory operations on the data: printing the schema, showing the first entry, and casting the raw data (sent as binary by Kafka) as a string.

In [3]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [4]:
raw_assessments.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
raw_assessments.show(1)

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
+----+--------------------+-----------+---------+------+--------------------+-------------+
only showing top 1 row



In [43]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))
assessments.show(1)

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
+--------------------+
only showing top 1 row



In [7]:
assessments.collect()[0]

Row(value='{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.

By printing the entire contents of the first row, we see that each row represents one assessment. Thus, counting the number of rows gives the total number of assessments in the dataset: 3280.

In [24]:
assessments.count()

3280

After casting the raw data into strings, we can save and read the data in parquet format, using the "overwrite" operation to avoid errors and complications resulting from the data already having been previously written.

In [8]:
assessments.write.parquet("/tmp/assessments", "overwrite")

In [9]:
read_assessments = spark.read.parquet('/tmp/assessments')
read_assessments.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



We can further convert the data from parquet format to RDD, the Spark equivalent of a DataFrame. This allows for some simple querying, but the nested data structures do not appear to suit this method well.

In [11]:
# Transform to DataFrame
from pyspark.sql.functions import from_json, col
json_schema = spark.read.json(read_assessments.rdd.map(lambda row: row.value)).schema
read_assessments = read_assessments.withColumn('json', from_json(col('value'), json_schema))
read_assessments.select(
    read_assessments.json.max_attempts,
    read_assessments.json.sequences,
    read_assessments.json.certification,
    read_assessments.json.exam_name
).show()

+-----------------+--------------------+------------------+--------------------+
|json.max_attempts|      json.sequences|json.certification|      json.exam_name|
+-----------------+--------------------+------------------+--------------------+
|              1.0|[1,[false,2,1,1,4...|             false|Normal Forms and ...|
|              1.0|[1,[false,1,2,1,4...|             false|Normal Forms and ...|
|              1.0|[1,[false,3,0,1,4...|             false|The Principles of...|
|              1.0|[1,[false,2,2,0,4...|             false|The Principles of...|
|              1.0|[1,[false,3,0,1,4...|             false|Introduction to B...|
|              1.0|[1,[true,5,0,0,5,...|             false|        Learning Git|
|              1.0|[1,[true,1,0,0,1,...|             false|Git Fundamentals ...|
|              1.0|[1,[true,5,0,0,5,...|             false|Introduction to P...|
|              1.0|[1,[true,4,0,0,4,...|             false|Intermediate Pyth...|
|              1.0|[1,[false

We manually define the schema for each assessment. Many of the fields will likely be unimportant, but we do not omit any here in case they do become useful.

In [50]:
assessments_schema = StructType([
    StructField('keen_timestamp', StringType(), True),
    StructField('max_attempts', StringType(), True),
    StructField('started_at', StringType(), True),
    StructField('base_exam_id', StringType(), True),
    StructField('user_exam_id', StringType(), True),
    StructField('keen_created_at', StringType(), True),
    StructField('certification', StringType(), True),
    StructField('keen_id', StringType(), True),
    StructField('exam_name', StringType(), True),
    StructField('sequences', StructType([
        StructField('attempt', StringType(), True),
        StructField('id', StringType(), True),
        StructField('questions', ArrayType(StructType([
            StructField('user_incomplete', BooleanType(), True),
            StructField('user_correct', BooleanType(), True),
            StructField('user_submitted', StringType(), True),
            StructField('id', StringType(), True),
            StructField('user_result', StringType(), True),
            StructField('options', ArrayType(StructType([
                StructField('checked', BooleanType(), True),
                StructField('at', StringType(), True),
                StructField('id', StringType(), True),
                StructField('submitted', StringType(), True),
                StructField('correct', BooleanType(), True)
            ]), False), True)
        ]), False)),
        StructField('counts', StructType([
            StructField('incomplete', IntegerType(), True),
            StructField('submitted', IntegerType(), True),
            StructField('incorrect', IntegerType(), True),
            StructField('all_correct', BooleanType(), True),
            StructField('correct', IntegerType(), True),
            StructField('total', IntegerType(), True),
            StructField('unanswered', IntegerType(), True)
        ]))
    ]))
])

Using the schema defined above, we can write the assessments data as an RDD with the correct structure. We then verify the schema by printing the schema and making simple queries.

In [51]:
struct_assessments = assessments.rdd.map(lambda x: json.loads(x.value)).toDF(schema=assessments_schema)

In [52]:
struct_assessments.printSchema()

root
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- base_exam_id: string (nullable = true)
 |-- user_exam_id: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- sequences: struct (nullable = true)
 |    |-- attempt: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- questions: array (nullable = true)
 |    |    |-- element: struct (containsNull = false)
 |    |    |    |-- user_incomplete: boolean (nullable = true)
 |    |    |    |-- user_correct: boolean (nullable = true)
 |    |    |    |-- user_submitted: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- user_result: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: st

In [53]:
struct_assessments.take(1)[0]['exam_name']

'Normal Forms and All That Jazz Master Class'

In [30]:
struct_assessments.take(1)[0]['sequences']['counts']['all_correct']

False

In [31]:
struct_assessments.take(1)[0]['sequences']['questions'][0]['options'][0]['checked']

True

We convert the correctly-structured RDD to a relational table, which allows for SQL queries. We test this with a few simple example queries.

In [32]:
struct_assessments.registerTempTable('assessments')

In [33]:
spark.sql("SELECT exam_name FROM assessments").show(10)

+--------------------+
|           exam_name|
+--------------------+
|Normal Forms and ...|
|Normal Forms and ...|
|The Principles of...|
|The Principles of...|
|Introduction to B...|
|        Learning Git|
|Git Fundamentals ...|
|Introduction to P...|
|Intermediate Pyth...|
|Introduction to P...|
+--------------------+
only showing top 10 rows



In [108]:
spark.sql("SELECT \
exam_name \
FROM assessments \
WHERE sequences.counts.all_correct = True").show(1)

+------------+
|   exam_name|
+------------+
|Learning Git|
+------------+
only showing top 1 row



## Answering Business Questions with Data Queries

**How many different exams are present in this dataset?**

By counting the number of unique exam IDs, we find that there are 107 unique exam IDs represented in this dataset. This likely corresponds to 107 unique exams, each with multiple attempts by different students.

In [19]:
spark.sql("SELECT COUNT(DISTINCT base_exam_id) FROM assessments").show()

+----------------------------+
|count(DISTINCT base_exam_id)|
+----------------------------+
|                         107|
+----------------------------+



**Do any exams have more than one occurrence?**

By counting the number of unique exam IDs per exam name, we find that only a few exams are represented multiple times in the dataset, while most exams occur only once.

In [109]:
spark.sql("SELECT \
exam_name, \
COUNT(DISTINCT base_exam_id) AS num_exams \
FROM assessments \
GROUP BY exam_name \
ORDER BY num_exams DESC").show(107, truncate = False)

+-----------------------------------------------------------------------+---------+
|exam_name                                                              |num_exams|
+-----------------------------------------------------------------------+---------+
|Introduction to Python                                                 |2        |
|Great Bash                                                             |2        |
|Architectural Considerations for Hadoop Applications                   |2        |
|Being a Better Introvert                                               |2        |
|Learning Data Modeling                                                 |1        |
|Networking for People Who Hate Networking                              |1        |
|Introduction to Java 8                                                 |1        |
|Learning Apache Hadoop                                                 |1        |
|Learning Spring Programming                                            |1  

When defining the schema, an error occurred if the "certification" field was mapped to a BooleanType. Apparently, the "certification" field apparently contains only "null" and "false".

In [56]:
spark.sql("SELECT \
certification, \
COUNT(*) \
FROM assessments \
GROUP BY certification").show(truncate = False)

+-------------+--------+
|certification|count(1)|
+-------------+--------+
|null         |132     |
|false        |3148    |
+-------------+--------+



**What was the most popular class?**

We group assessments by exam ID and exam name, in order to avoid double-counting the exams that appear twice in the dataset. Assuming that the number of exams for a class exactly equals the number of students taking the class, the most popular class was "Learning Git", with 394 students.

In the rest of this report, we explore the "Learning Git" assessments with more in-depth queries. However, note that these are examples that are applicable to all exams.

In [110]:
spark.sql("SELECT \
COUNT(*) AS num, \
exam_name, \
base_exam_id \
FROM assessments \
GROUP BY base_exam_id, exam_name \
ORDER BY num DESC").show(107, truncate = False)

+---+-----------------------------------------------------------------------+------------------------------------+
|num|exam_name                                                              |base_exam_id                        |
+---+-----------------------------------------------------------------------+------------------------------------+
|394|Learning Git                                                           |8b4488de-43a5-4ffa-bf82-af1e19ee1b64|
|158|Introduction to Java 8                                                 |41858ac3-1394-451b-bf7c-c10f52034a9a|
|158|Intermediate Python Programming                                        |1a233da8-e6e5-48a6-8c3c-806e312cce12|
|128|Learning to Program with R                                             |b114e4a4-a192-4dff-a5cd-8e7782bb1623|
|122|Introduction to Python                                                 |7e2e0b53-a7ba-458d-8bc6-356f8dea8815|
|119|Introduction to Machine Learning                                       |c46

**How many students got every question correct on the "Learning Git" exam?**

Each assessment contains a record of whether the student got all questions correct. We find that 130 students got every question correct.

In [111]:
spark.sql("SELECT \
COUNT(*) AS num_students \
FROM assessments \
WHERE sequences.counts.all_correct = True \
AND exam_name = 'Learning Git'").show()

+------------+
|num_students|
+------------+
|         130|
+------------+



**What was the average score on the "Learning Git" exam?**

Each assessment contains a record of the number of questions the student answered correctly and the total number of questions. Summing these over all students gives an average score of 1332/1970 = 67.6%.

In [112]:
spark.sql("SELECT \
SUM(sequences.counts.correct) AS num_correct_answers, \
SUM(sequences.counts.total) AS num_total_answers \
FROM assessments \
WHERE exam_name = 'Learning Git'").show()

+-------------------+-----------------+
|num_correct_answers|num_total_answers|
+-------------------+-----------------+
|               1332|             1970|
+-------------------+-----------------+



**What were the easiest and hardest questions on the "Learning Git" exam?**

We assume the difficulty of a question is related to the proportion of students who got it wrong. The fewer students got a question wrong, the easier the question was, and vice versa. By counting the number of assessments with correct answers for each question, we find that questions 1 and 2 were the easiest, with 270/394 = 68.5% of students answering correctly, while question 3 was the hardest, with 261/394 = 66.2% of students answering correctly.

In [113]:
spark.sql("SELECT \
SUM(CASE sequences.questions[0].user_correct WHEN True THEN 1 ELSE 0 END) AS q1_correct, \
SUM(CASE sequences.questions[1].user_correct WHEN True THEN 1 ELSE 0 END) AS q2_correct, \
SUM(CASE sequences.questions[2].user_correct WHEN True THEN 1 ELSE 0 END) AS q3_correct, \
SUM(CASE sequences.questions[3].user_correct WHEN True THEN 1 ELSE 0 END) AS q4_correct, \
SUM(CASE sequences.questions[4].user_correct WHEN True THEN 1 ELSE 0 END) AS q5_correct \
FROM assessments \
WHERE exam_name = 'Learning Git'").show()

+----------+----------+----------+----------+----------+
|q1_correct|q2_correct|q3_correct|q4_correct|q5_correct|
+----------+----------+----------+----------+----------+
|       270|       270|       261|       264|       267|
+----------+----------+----------+----------+----------+



**On question 3 of the "Learning Git" exam, how many answer choices were there?**

There were 4 answer choices for question 3 of the "Learning Git" exam.

In [99]:
spark.sql("SELECT \
size(sequences.questions[2].options) AS num_choices \
FROM assessments \
WHERE exam_name = 'Learning Git'").show(1)

+-----------+
|num_choices|
+-----------+
|          4|
+-----------+
only showing top 1 row



**On question 3 of the "Learning Git" exam, which incorrect answer choice was chosen most frequently?**

Apparently, within each question the correct answer choice is randomized. Each answer choice only contains an id, and there is no way to determine the contents of each answer choice from this data. However, we can still ask a different question:

**On question 3 of the "Learning Git" exam, how often was each answer choice the correct one?**

Option 2 was the correct choice substantially more often than the others. With perfect randomization, we should expect each answer choice to be the correct one about 394/4 = 99 times.

In [114]:
spark.sql("SELECT \
SUM(CASE sequences.questions[2].options[0].correct WHEN True THEN 1 ELSE 0 END) AS opt1_correct, \
SUM(CASE sequences.questions[2].options[1].correct WHEN True THEN 1 ELSE 0 END) AS opt2_correct, \
SUM(CASE sequences.questions[2].options[2].correct WHEN True THEN 1 ELSE 0 END) AS opt3_correct, \
SUM(CASE sequences.questions[2].options[3].correct WHEN True THEN 1 ELSE 0 END) AS opt4_correct \
FROM assessments \
WHERE exam_name = 'Learning Git'").show()

+------------+------------+------------+------------+
|opt1_correct|opt2_correct|opt3_correct|opt4_correct|
+------------+------------+------------+------------+
|         101|         121|          91|          81|
+------------+------------+------------+------------+

